Author: Joshua
Summary: Run a single instance of the two stage stochastic program
Input:
Output:

In [1]:
using Pkg
Pkg.add("Pkg")
Pkg.add("Dates")

joshpath = "C:/Users/Owner/Documents/Austin/AmbulanceDeployment/AmbulanceDeployment.jl-legacy"

PROJECT_ROOT = joshpath
currentpath = joshpath

cd(currentpath)
Pkg.activate(".")

#include(".//src//problem.jl")
#include(".//src//model.jl")

using AmbulanceDeployment 
using DataFrames, Winston, JLD, CSV, Gurobi, JuMP, GLPK, Dates

   Updating registry at `C:\Users\Owner\.julia\registries\General`
  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`
 Activating environment at `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
┌ Info: Precompiling AmbulanceDeployment [d2c08a37-75f3-4934-bcbc-64feba624d07]
└ @ Base loading.jl:1278
┌ Warning: Package AmbulanceDeployment does not have Random in its dependencies:
│ - If you have AmbulanceDeployment checked out for development and have
│   added Random as a dependency but hav

LoadError: [91mFailed to precompile AmbulanceDeployment [d2c08a37-75f3-4934-bcbc-64feba624d07] to C:\Users\Owner\.julia\compiled\v1.5\AmbulanceDeployment\7u3Yr_ZG3xL.ji.[39m

In [2]:
# hourly calls - regions x hours x number of calls/ per region per hour (e.g. 210 x 10000 x Z)
#adjacent_nbhd - boolean matrix, regionxregion (210x210) - if neighborhoods/region are adjacent?
#coverage - boolean matrix, regions x stations (210x44) - true if station is within coverage of region
#incidents - god object
hourly_calls = CSV.File(PROJECT_ROOT * "/test/austin-data/Full_WeekdayCalls.csv") |> DataFrame
# weekend_hourly_calls = CSV.File("data/processed/2-weekend_calls.csv") |> DataFrame
adjacent_nbhd = CSV.File(PROJECT_ROOT * "/test/austin-data/adjacent_nbhd.csv") |> DataFrame
coverage = CSV.read(PROJECT_ROOT * "/test/austin-data/coverage_real.csv", DataFrame, header=false)
coverage = convert(Array{Bool, 2}, coverage[:, :])
incidents = CSV.File(PROJECT_ROOT * "/test/austin-data/austin_incidents.csv") |> DataFrame

LoadError: [91mUndefVarError: CSV not defined[39m

In [3]:
#regions - list of regions/neighborhoods where demands come from (1 ... 210)
#locations -list of stations (1 ... 44)
#demands - hours x regions = demands
regions = Int[parse(Int,string(x)) for x in names(hourly_calls[:,6:ncol(hourly_calls)])]
locations = collect(1:size(coverage,2))
adjacent = convert(Array, adjacent_nbhd[:,2:ncol(adjacent_nbhd)])[regions,regions] .> 0.5
demand = convert(Array,hourly_calls[:,6:end]);

LoadError: [91mUndefVarError: ncol not defined[39m

In [4]:
#filter incidents
incidents = incidents[.~ismissing.(incidents[!,:stn1_min]), :] # drop 44 calls that were "unreachable" (because all stations are reachable from each other)
#remove "faulty data" if events in the log do not come in order
incidents[!,:interarrival_seconds] = [0; incidents[!,:arrival_seconds][2:end] - incidents[!,:arrival_seconds][1:end-1]]
incidents[!,:isweekday] .= true
incidents[!,:isweekday][incidents[!,:dow] .== "Sun"] .= false
incidents[!,:isweekday][incidents[!,:dow] .== "Sat"] .= false;
incidents = incidents[incidents[!,:isweekday],:]

regions2index = Dict{Int,Int}(regions[i]=>i for i in 1:length(regions))

LoadError: [91mUndefVarError: incidents not defined[39m

In [5]:
#= previously there was an error that didnt account for all the regions because it
 it only included regions for incidents. the regions without incidents are changed
 to 0=#
for x in incidents[!,:neighborhood]
    if(!haskey(regions2index,x))
        regions2index[x] = 0
    end
end
incidents[!,:neighborhood] = [regions2index[x] for x in incidents[!,:neighborhood]];

calls = incidents[:,[[:hour,:dow,:month,:year,:neighborhood,:interarrival_seconds];
                     Symbol[Symbol("stn$(i)_min") for i in locations]]]

DataFrames.first(calls, 6)

LoadError: [91mUndefVarError: incidents not defined[39m

In [6]:
# We focus on emergency calls during the "peak period" (8AM - 8PM),
# with the emergency calls from the first 3 month as our training set,
# and the subsequent emergency calls from the remaining months as our test set
peak_period = (hourly_calls[!,:hour] .>= 8) .* (hourly_calls[!,:hour] .<= 20)
indices = 1:DataFrames.nrow(hourly_calls);
train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
test_filter  = .~train_filter;
train_indices = indices[train_filter]
test_indices = indices[test_filter];

#will change the 30 later
p = DeploymentProblem(30, length(locations), length(regions), demand, train_indices,
                      test_indices, coverage[regions,:], Array{Bool,2}(adjacent));

test_inc_peak = inc_peak_period .* inc_test_filter
test_inc_offpeak = .~inc_peak_period .* inc_test_filter;

LoadError: [91mUndefVarError: hourly_calls not defined[39m

In [7]:
amb_deployment = Dict{Symbol, Dict{Int, Vector{Int}}}()  

Dict{Symbol,Dict{Int64,Array{Int64,1}}}()

In [8]:
(deployment_model, name) = (next_dp -> StochasticDeployment(next_dp, nperiods=500), :Stochastic)
println("$name: ")
amb_deployment[name] = Dict{Int, Vector{Int}}()
namb = 50
println("$namb ")
p.nambulances = namb
println("time for model generation - stochastic $namb ambulances")
## make sure to include @time begin
@time model = deployment_model(p)
set_optimizer(model.m, Gurobi.Optimizer)
println("time for model solution - stochastic $namb ambulances")
@time AmbulanceDeployment.optimize!(model, p)


Stochastic: 
50 


LoadError: [91mUndefVarError: p not defined[39m

In [9]:
amb_deployment[name][namb] = deployment(model)
println

LoadError: [91mUndefVarError: deployment not defined[39m